In [23]:
from ipywidgets import interact, fixed
from IPython.display import display
from drawdata import draw_scatter
import matplotlib.pyplot as plt
import ipywidgets as widgets
import pandas as pd
import osmnx as ox
import notutils
import pods
import mlai

In [6]:
draw_scatter()

In [7]:
# I have some weird terminal settings so Pandas' built-in read_clipboard method fails
# So I hacked together my own
def read_clipboard():
  from tkinter import Tk
  from io import StringIO
  with StringIO(Tk().clipboard_get()) as buf:
    return pd.read_csv(buf, sep=",")

In [8]:
drawdata_data = read_clipboard()
drawdata_data.head()

,x,y,z
0,28.494964,129.077291,a
1,35.634806,160.142493,a
2,48.928444,162.233873,a
3,32.396299,145.528011,a
4,-15.822448,133.136550,a


In [9]:
drawdata_data[["x", "y"]].corr() # Positive value => positive correlation

,x,y
x,1.000000,0.820008
y,0.820008,1.000000


In [10]:
drawdata_data.groupby(["z"])[["x", "y"]].corr() # Negative values => negative correlation

x         y
z                      
a x  1.000000 -0.756157
  y -0.756157  1.000000
b x  1.000000 -0.805574
  y -0.805574  1.000000

In [11]:
data_interact = drawdata_data

In [12]:
column = "z"
items = ["all"] + sorted(data_interact[column].unique().tolist())

In [13]:
def view_df(dataframe, column, item):
  display(dataframe[dataframe[column]==item] if item != "all" else dataframe)

interact(
  view_df,
  dataframe=fixed(data_interact), 
  column=fixed(column),
  item=widgets.Select(options=items)
);

,x,y,z
0,28.494964,129.077291,a
1,35.634806,160.142493,a
2,48.928444,162.233873,a
3,32.396299,145.528011,a
4,-15.822448,133.136550,a
...,...,...,...
221,456.890444,473.998989,b
222,417.300476,505.259410,b
223,440.496872,510.962922,b
224,402.457897,472.168971,b


In [14]:
def view_df2(dataframe, column, item, number):
  display((dataframe[dataframe[column]==item] if item != "all" else dataframe)[:number])

interact(
  view_df2,
  dataframe=fixed(data_interact), 
  column=fixed(column),
  item=widgets.Select(options=items),
  number=widgets.IntSlider(min=0, max=15, step=1, value=5)
);

,x,y,z
0,28.494964,129.077291,a
1,35.634806,160.142493,a
2,48.928444,162.233873,a
3,32.396299,145.528011,a
4,-15.822448,133.136550,a


In [15]:
def view_df3(dataframe, column, item, number, filterColumn, filterThreshold):
  df = (dataframe[dataframe[column]==item] if item != "all" else dataframe)
  display(df[df[filterColumn] <= filterThreshold][:number])

filterColumns = ["x", "y"]

interact(
  view_df3,
  dataframe=fixed(data_interact), 
  column=fixed(column),
  item=widgets.Select(options=items),
  number=widgets.IntSlider(min=0, max=15, step=1, value=5),
  filterColumn=filterColumns,
  filterThreshold=widgets.FloatSlider(
      min=data_interact[filterColumns].min().min(),
      max=data_interact[filterColumns].max().max(),
      value=data_interact[filterColumns].max().max())
);

,x,y,z
0,28.494964,129.077291,a
1,35.634806,160.142493,a
2,48.928444,162.233873,a
3,32.396299,145.528011,a
4,-15.822448,133.136550,a


In [17]:
place_name = "Kampala, Uganda"

latitude = 0.347596 # Kampala latitude
longitude = 32.582520 # Kampala longitude
placestub = place_name.lower().replace(' ', '-').replace(',','')

In [18]:
box_width = 0.02 # About 2.2 km
box_height = 0.02
north = latitude + box_height/2
south = latitude - box_height/2
west = longitude - box_width/2
east = longitude + box_width/2

In [19]:
# Retrieve POIs
tags = {"amenity": True, 
        "buildings": True, 
        "historic": True, 
        "leisure": True, 
        "shop": True, 
        "tourism": True}

In [24]:
pois = ox.geometries_from_bbox(north, south, east, west, tags)

In [25]:
print("There are {number} points of interest surrounding {placename} latitude: {latitude}, longitude: {longitude}".format(number=len(pois), placename=place_name, latitude=latitude, longitude=longitude))

There are 844 points of interest surrounding Kampala, Uganda latitude: 0.347596, longitude: 32.58252


In [26]:
pois.info

<bound method DataFrame.info of                              amenity  cuisine  \
element_type osmid                              
node         599171967    restaurant  mexican   
             599171968           NaN      NaN   
             599171969           NaN      NaN   
             599171971      pharmacy      NaN   
             601370312          fuel      NaN   
...                              ...      ...   
way          749930994       parking      NaN   
             856274788  kindergarten      NaN   
             856274789  kindergarten      NaN   
             858227179           NaN      NaN   
             932375082           NaN      NaN   

                                                   name            phone  \
element_type osmid                                                         
node         599171967                         Que Pasa  +256 783 874469   
             599171968           Millennium Supermarket              NaN   
             599171969    

In [27]:
keys = ["name",
        "addr:city",
        "addr:postcode",
        "amenity",
        "building",
        "building:name",
        "building:colour",
        "building:material",
        "historic",
    "memorial",
        "religion",
        "tourism",
        "emergency",
        "leisure",
        "shop"]

present_keys = [key for key in keys if key in pois.columns]
pois[present_keys]

name addr:city  \
element_type osmid                                                  
node         599171967                         Que Pasa       NaN   
             599171968           Millennium Supermarket       NaN   
             599171969                         Checkers       NaN   
             599171971          Makro Pharmacy (U) Ltd.       NaN   
             601370312                            Kobil       NaN   
...                                                 ...       ...   
way          749930994                              NaN       NaN   
             856274788  Early childhood learning center       NaN   
             856274789  Early childhood learning center       NaN   
             858227179                       Salon city       NaN   
             932375082                              NaN       NaN   

                       addr:postcode       amenity building religion tourism  \
element_type osmid                                                             
node         599171967           NaN    restaurant      NaN      NaN     NaN   
             599171968           NaN           NaN      NaN      NaN     NaN   
             599171969           NaN           NaN      NaN      NaN     NaN   
             599171971           NaN      pharmacy      NaN      NaN     NaN   
             601370312           NaN          fuel      NaN      NaN     NaN   
...                              ...           ...      ...      ...     ...   
way          749930994           NaN       parking      NaN      NaN     NaN   
             856274788           NaN  kindergarten      yes      NaN     NaN   
             856274789           NaN  kindergarten      yes      NaN     NaN   
             858227179           NaN           NaN      yes      NaN     NaN   
             932375082           NaN           NaN      yes      NaN     NaN   

                       emergency leisure         shop  
element_type osmid                                     
node         599171967       NaN     NaN          NaN  
             599171968       NaN     NaN  supermarket  
             599171969       NaN     NaN  supermarket  
             599171971       NaN     NaN          NaN  
             601370312       NaN     NaN          NaN  
...                          ...     ...          ...  
way          749930994       NaN     NaN          NaN  
             856274788       NaN     NaN          NaN  
             856274789       NaN     NaN          NaN  
             858227179       NaN     NaN  hairdresser  
             932375082       NaN   pitch          NaN  

[844 rows x 10 columns]

In [29]:
graph = ox.graph_from_bbox(north, south, east, west)
nodes, edges = ox.graph_to_gdfs(graph)
area = ox.geocode_to_gdf(place_name)
tourist_places = pois[pois.tourism.notnull()]

In [49]:
tourist_places.loc["node"]

,amenity,cuisine,name,phone,website,geometry,shop,brand,brand:wikidata,brand:wikipedia,...,community_centre,denomination,bus,public_transport,building:levels,multipurpose,contact:twitter,rooms,guest_house,smoking
osmid,,,,,,,,,,,,,,,,,,,,,
3032234586,NaN,NaN,Fat Cat,+256 771 393892,http://www.fatcatkampala.com/,POINT (32.58834 0.33994),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4119308804,mobile_money_agent,NaN,Green apple,NaN,NaN,POINT (32.58817 0.34264),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4942954054,NaN,NaN,Okrut Close,NaN,NaN,POINT (32.59180 0.33868),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5678162422,NaN,NaN,Tagore Apartments,NaN,NaN,POINT (32.58474 0.33914),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7743212629,NaN,NaN,NaN,NaN,NaN,POINT (32.58812 0.34466),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8445011017,NaN,NaN,Prisma Hotel,0414371122,NaN,POINT (32.57987 0.34816),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8456545033,NaN,NaN,Silverline Condos,NaN,NaN,POINT (32.58329 0.34251),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8456545129,NaN,NaN,Kida Guesthouse,NaN,NaN,POINT (32.57336 0.34773),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8481722617,NaN,NaN,Koda Guest House,NaN,NaN,POINT (32.57340 0.34824),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
for i in tourist_places["geometry"]["way"]:
    print("latitude: {latitude}, longitude: {longitude}".format(latitude=i.centroid.y, longitude=i.centroid.x))

latitude: 0.3376459150435436, longitude: 32.58405714602133
latitude: 0.34778169999999997, longitude: 32.5727817


In [125]:
def exercise_3(place_name, latitude, longitude, box_width=0.02, box_height=0.02, match_threshold_dps=3):
  placestub = place_name.lower().replace(' ', '-').replace(',','')
  north = latitude + box_height/2
  south = latitude - box_height/2
  west = longitude - box_width/2
  east = longitude + box_width/2
  pois = ox.geometries_from_bbox(north, south, east, west, {"healthcare": True})
  
  lng = []
  lat = []
  if pois.shape[0] > 0:
    for i in pois["geometry"]:
      lng.append(i.centroid.x)
      lat.append(i.centroid.y)
  
  pois_latitude = pd.Series(lat, dtype=float)
  pois_longitude = pd.Series(lng, dtype=float)

  pods_data = pods.datasets.nigeria_nmis()["Y"]
  pods_data = pods_data[south <= pods_data["latitude"]]
  pods_data = pods_data[north >= pods_data["latitude"]]
  pods_data = pods_data[west <= pods_data["longitude"]]
  pods_data = pods_data[east >= pods_data["longitude"]]

  pois_latitude_approx = round(pois_latitude, match_threshold_dps)
  pois_longitude_approx = round(pois_longitude, match_threshold_dps)
  pois["coords_approx"] = list(pois_latitude_approx.astype(str) + "," + pois_longitude_approx.astype(str))

  pods_data_latitude_approx = round(pods_data["latitude"], match_threshold_dps)
  pods_data_longitude_approx = round(pods_data["longitude"], match_threshold_dps)
  pods_data["coords_approx"] = pods_data_latitude_approx.astype(str) + "," + pods_data_longitude_approx.astype(str)

  df = pois.set_index("coords_approx").join(pods_data.set_index("coords_approx"), how="outer")

  return df[~(df["geometry"].duplicated(keep='first', ) & df["geometry"].notna())]

place_name = "Lagos, Nigeria"

latitude = 6.5244
longitude = 3.3792

exercise_3(place_name, latitude, longitude, box_width=0.34, box_height=0.34)

,amenity,fixme,healthcare,name,geometry,dispensing,opening_hours,shop,beds,electricity,...,antenatal_care_yn,family_planning_yn,malaria_treatment_artemisinin,sector,formhub_photo_id,gps,survey_id,unique_lga,latitude,longitude
coords_approx,,,,,,,,,,,,,,,,,,,,,
"6.414,3.252",NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,True,True,True,health,1339150079160.jpg,6.41354952 3.25234932 30.899999618530273 5.0,6a1fa574-e575-4f38-b8a9-b621614118f0,lagos_amuwo_odofin,6.413550,3.252349
"6.418,3.33",NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,True,False,True,health,1339160051655.jpg,6.41791793 3.32977637 31.899999618530273 5.0,1708feeb-c6fc-4fc9-a0a5-646d785ed3dc,lagos_amuwo_odofin,6.417918,3.329776
"6.424,3.453",NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,True,True,True,health,1398437009514.jpg,6.42443336 3.45259356 27.200000762939453 5.0,db0267c3-05af-4cf2-9b68-3f57897d935b,lagos_eti_osa,6.424433,3.452594
"6.425,3.428",NaN,NaN,laboratory,SynLab,"LINESTRING (3.42783 6.42555, 3.42780 6.42548, ...",NaN,Mo-Su 08:00-20:00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"6.427,3.347",NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,False,False,False,health,1339061704389.jpg,6.42700765 3.34656164 23.0 5.0,6898a17b-de52-4bfb-9472-e953caf8b858,lagos_apapa,6.427008,3.346562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"6.694,3.236",NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,True,True,True,health,1399383108295.jpg,6.69374147 3.23594502 80.9000015258789 5.0,2c1772b5-fded-453a-a264-014be279aeb9,ogun_ado_odo_ota,6.693741,3.235945
"6.694,3.238",NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,True,True,True,health,1399377064590.jpg,6.6937601 3.237553 96.30000305175781 5.0,25a23274-7f9e-4519-8aa1-16d4786d5340,ogun_ado_odo_ota,6.693760,3.237553
"6.694,3.238",NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,True,True,True,health,1338474054558.jpg,6.69379286 3.23762772 84.69999694824219 5.0,33d9b128-ac98-4862-8feb-cb57be3597ca,ogun_ado_odo_ota,6.693793,3.237628


In [126]:
place_name = "Abuja, Nigeria"

latitude = 9.05785
longitude = 7.49508

exercise_3(place_name, latitude, longitude, box_width=0.42, box_height=0.42)

,amenity,healthcare,name,geometry,addr:street,emergency,operator,operator:type,source,staff_count:doctors,...,antenatal_care_yn,family_planning_yn,malaria_treatment_artemisinin,sector,formhub_photo_id,gps,survey_id,unique_lga,latitude,longitude
coords_approx,,,,,,,,,,,,,,,,,,,,,
"8.878,7.648",NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,True,True,True,health,1393481563713.jpg,8.87843536 7.64756047 324.20001220703125 5.0,c4371597-6d15-46df-b0dd-ba97891470e7,nasarawa_karu,8.878435,7.647560
"8.879,7.647",NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,True,True,True,health,1342345377299.jpg,8.87871215 7.64698805 316.29998779296875 5.0,b7491e39-f38d-4a60-814c-f6560aab9f09,nasarawa_karu,8.878712,7.646988
"8.881,7.651",NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,True,True,False,health,1342348087942.jpg,8.88094485 7.65111891 319.8999938964844 5.0,acbba073-855e-4d45-8835-8904bcaa0d32,nasarawa_karu,8.880945,7.651119
"8.889,7.442",NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,True,True,True,health,1398255846323.jpg,8.888966666666667 7.442024999999999 420.0 5.0,3d394ea9-7102-4b56-a564-91476c912cdc,fct_municipal_area_council,8.888967,7.442025
"8.893,7.659",NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,True,True,True,health,1393480093260.jpg,8.89347025 7.65883824 312.3999938964844 5.0,40075f36-0237-4974-9af7-3d4b46513762,nasarawa_karu,8.893470,7.658838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"9.248,7.438",NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,True,True,True,health,1396539072631.jpg,9.248211666666666 7.437751666666666 729.1 4.9,72f02935-d1cf-4844-9e75-07179820c9c7,fct_bwari,9.248212,7.437752
"9.251,7.298",NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,True,health,1347098723990.jpg,9.25121313 7.29840238 548.0 5.0,1399d03c-5e54-4fc1-b28d-12050628a493,niger_tafa,9.251213,7.298402
"9.256,7.388",NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,True,True,True,health,1305704886414.jpg,9.256324768066406 7.387978434562683 536.0 3.0,7927b276170874bac8e3924ecca9a4c7,fct_bwari,9.256325,7.387978
